## Lab Assignment 3 Scientific Computing

Nick Boon & Marleen Rijksen

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.sparse import spdiags, diags
from scipy.sparse.linalg import eigs, eigsh
from scipy.linalg import eig, eigh
from scipy.linalg import solve
from scipy.sparse.linalg import spsolve
from mpl_toolkits.mplot3d.axes3d import Axes3D

from numba import jit

from matplotlib import animation, rc

from IPython.display import HTML
rc('animation', html='html5')

from sklearn.preprocessing import normalize

In [ ]:
%matplotlib inline

# 3.1 Eigenmodes of drums or membranes of different shapes

In [ ]:
#from https://stackoverflow.com/a/35126679
def set_aspect_equal_3d(ax):
    """
    Fix equal aspect bug for 3D plots in X and Y.    
    """

    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()

    xmean = np.mean(xlim)
    ymean = np.mean(ylim)

    plot_radius = np.max([
        abs(lim - mean_) for lims, mean_ in ((xlim, xmean), (ylim, ymean))
        for lim in lims
    ])

    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])

def laplacian(shape, dx):
    """
    Construct a sparse matrix that applies the 5-point discretization of
    the Laplacian operator

    Input:
    shape - tuple of matrix shape (y,x)
    dx - discretization size

    Output:
    Sparse matrix that evaluates the 5-point discretization of a shape-sized matrix
    """
    
    M, N = shape
    e = np.ones(N * M)
    e2 = ([1] * (N - 1) + [0]) * M
    e3 = ([0] + [1] * (N - 1)) * M

    A = spdiags([-4 * e, e2, e3, e, e], [0, -1, 1, -N, N], N * M, N * M)

    return A / (dx ** 2)

def circular(r):
    """
    Fill matrix with circle of size r
    """
    circle = np.zeros((r,r))
    for i,x in enumerate(np.arange(-r/2+0.5,r/2+0.5)):
        for j,y in enumerate(np.arange(-r/2+0.5,r/2+0.5)):
            if np.sqrt(x ** 2 + y ** 2) <= r/2:
                circle[j,i] = 1
    return circle

def laplacian_circle(r, dx):
    """
    Creates the laplacian matrix from a matrix containing
    ones for sites part of the domain under consideration
    
    Input
    r - size of matrix returned
    dx - discretization size
    """
    circle = circular(r)

    #also non-zero for sites not part of the circle!
    #matrix is singular otherwise
    self = np.ones(r * r) 
    
    left = []
    right = []
    up = []
    down = []
    
    #iterate over all sites and check whether the current
    #site and neighbour are part of the circle
    for j in range(r):
        for i in range(r):
            if i == 0:
                left.append(0)
            else:
                if circle[j,i-1] == 1 and circle[j,i] == 1:
                    left.append(1)
                else:
                    left.append(0)
            if i == r-1:
                right.append(0)
            else:
                if circle[j,i+1] == 1 and circle[j,i] == 1:
                    right.append(1)
                else:
                    right.append(0)
            if j == 0:
                up.append(0)
            else:
                if circle[j-1,i] == 1 and circle[j,i] == 1:
                    up.append(1)
                else:
                    up.append(0)
            if j == r-1:
                down.append(0)
            else:
                if circle[j+1,i] == 1 and circle[j,i] == 1:
                    down.append(1)
                else:
                    down.append(0)

    A = spdiags([-4 * self, left, right, up, down],\
                [0, 1, -1, r, -r], r ** 2, r ** 2)
    
    return A / (dx ** 2)

In [ ]:
def show_eigenmodes(L, shape, eigenmodes, membrane_shape='square'):
    """
    Shows eigenmodes smalles eigenmodes. Can use 'square'-shaped
    or 'circle'-shaped membrane
    
    Input
    L - length (y)/ diameter of space
    shape - shape of discretized space (y,x)
    eigenmodes - number of eigenmodes to show
    membrane-shape shape of the membrane, can be 'square' or 'circle'
    """
    dx = L / shape[0]
    k = eigenmodes

    if membrane_shape == 'square':
        a = eigs(laplacian(shape,dx), which='SM', k=k)
    elif membrane_shape == 'circle':
        a = eigs(laplacian_circle(shape[0],dx), which='SM', k=k)
    else:
        raise ValueError("Membrane shape can only be 'square' or 'circle'.")

    idx = np.abs(a[0]).argsort()[0:eigenmodes]
    eig_val = a[0][idx]
    eig_vec = a[1][:,idx]
    
    #create figure
    #every row has width 8 and height 5
    fig = plt.figure(figsize=(8, 5 * len(idx)))

    for i in range(eigenmodes):
        # set up the axes
        ax = fig.add_subplot(len(idx), 1, i + 1, projection='3d')

        # create surface
        if membrane_shape=='square':
            x = np.arange(0, L * shape[1]/shape[0], dx)
            y = np.arange(0, L, dx)
        elif membrane_shape=='circle':
            x = np.arange(-L/2,L/2,dx)
            y = np.arange(-L/2,L/2,dx)
        x, y = np.meshgrid(x, y)

        z = eig_vec[:, i].reshape(shape).real
        #frequency is 1/(c\lambda)=1/(c sqrt(-K))

        surf = ax.plot_surface(x, y, z, cmap=cm.viridis, rstride=3, cstride=3)
        cb = fig.colorbar(surf)
        cb.set_label("amplitude")
        ax.set_title("eigenvalue: %.2e\nfrequency: %.2f Hz" %
                     (eig_val[i].real, np.sqrt(-eig_val[i].real)))
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.set_zlabel("amplitude")
        set_aspect_equal_3d(ax)
    fig.tight_layout()

def get_eigenfrequencies(L,shape,eigenmodes,membrane_shape):
    dx = L / shape[0]
    k = eigenmodes

    if membrane_shape == 'square':
        a = eigs(laplacian(shape,dx), which='SM', k=k)
    elif membrane_shape == 'circle':
        a = eigs(laplacian_circle(shape[0],dx), which='SM', k=k)
    else:
        raise ValueError("Membrane shape can only be 'square' or 'circle'.")
    
    idx = np.abs(a[0]).argsort()[0:eigenmodes]
    eig_freq = sorted(np.sqrt(-a[0][idx]).real)
    eig_mod = a[1][:,idx].real
    
    return eig_freq, eig_mod

### Show eigenmodes in 3D plot

In [ ]:
show_eigenmodes(1,(100,100),10,'square')

In [ ]:
show_eigenmodes(1,(100,200),10,'square')

In [ ]:
show_eigenmodes(1,(100,100),10,'circle')

##### Show spectrum of eigenfrequencies depending on L

Show figure comparing each shape. Include small perturbation from real num_disc, as eigenfrequencies can overlap. Without the perturbation, this is not visible

In [ ]:
fig, ax = plt.subplots()
num_disc = []
eig_freq = []

num_eigenmodes = 5

for i in np.arange(1,5,0.5):
    a,_ = get_eigenfrequencies(i,(100,100),num_eigenmodes,'square')
    num_disc.extend([i + 0.1 for j in range(num_eigenmodes)])
    eig_freq.extend(a)
ax.scatter(num_disc,eig_freq,s=8,c='b',label='square L x L',alpha=0.3)

num_disc = []
eig_freq = []

for i in np.arange(1,5,0.5):
    a,_ = get_eigenfrequencies(i,(100,200),num_eigenmodes,'square')
    num_disc.extend([i for j in range(num_eigenmodes)])
    eig_freq.extend(a)
ax.scatter(num_disc,eig_freq,s=8,c='r',label='square 2L x L',alpha=0.3)

num_disc = []
eig_freq = []

for i in np.arange(1,5,0.5):
    a,_ = get_eigenfrequencies(i,(100,100),num_eigenmodes,'circle')
    num_disc.extend([i - 0.1 for j in range(num_eigenmodes)])
    eig_freq.extend(a)
ax.scatter(num_disc,eig_freq,s=8,c='g',label='circle diameter L',alpha=0.3)

ax.set_xlabel("Length L")
ax.set_ylabel("Eigenfrequency in Hz")
ax.set_title("Eigenfrequency as function of L\nNumber of eigenmodes: %d"%(num_eigenmodes))
ax.legend();

##### Show animation for square domain

In [ ]:
%%capture

L = 1
eigenmode = 0
shape = (20,20)
dx = 1 / shape[0]

freq, A = get_eigenfrequencies(1,shape,eigenmode+1,'square')
A = A[:,eigenmode].reshape(shape)
freq = freq[eigenmode]

max_amp = np.max(np.abs(A))

# create surface
x = np.arange(0, L * shape[1]/shape[0], dx)
y = np.arange(0, L, dx)
x, y = np.meshgrid(x, y)

def animate(i):
    global A, t, x, y
    t += TIME_STEP
    T = A * np.cos(freq * t)
    ax.clear()
    ax.plot_surface(x, y, T, cmap=cm.viridis,rstride=1, cstride=1)
    ax.set_zlim3d(-1.2*max_amp,1.2*max_amp)
    ax.set_title("Time: %.2f" % (t))
    set_aspect_equal_3d(ax)
    return ax

# set some parameters
FRAME_INTERVAL = 10
TIME_STEP = 0.005
NUM_FRAMES = int(2 * np.pi / (freq * TIME_STEP))
t = 0

fig = plt.figure(figsize=(6, 6))
# set up the axes
ax = fig.add_subplot(1, 1, 1, projection='3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("amplitude")
ani = animation.FuncAnimation(
    fig, animate, frames=NUM_FRAMES, interval=FRAME_INTERVAL)

In [ ]:
ani

##### Show animation for circular domain

In [ ]:
%%capture

L = 1
eigenmode = 0
shape = (20,20)
dx = 1 / shape[0]

freq, A = get_eigenfrequencies(1,shape,eigenmode+1,'circle')
A = A[:,eigenmode].reshape(shape)
freq = freq[eigenmode]

max_amp = np.max(np.abs(A))

# create surface
x = np.arange(-L/2, L/2, dx)
y = np.arange(-L/2, L/2, dx)
x, y = np.meshgrid(x, y)

def animate(i):
    global A, t, x, y
    t += TIME_STEP
    T = A * np.cos(freq * t)
    ax.clear()
    ax.plot_surface(x, y, T, cmap=cm.viridis,rstride=1, cstride=1)
    ax.set_zlim3d(-1.2*max_amp,1.2*max_amp)
    ax.set_title("Time: %.2f" % (t))
    set_aspect_equal_3d(ax)
    return ax

# set some parameters
FRAME_INTERVAL = 10
TIME_STEP = 0.005
NUM_FRAMES = int(2 * np.pi / (freq * TIME_STEP))
t = 0

fig = plt.figure(figsize=(6, 6))
# set up the axes
ax = fig.add_subplot(1, 1, 1, projection='3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("amplitude")
ani = animation.FuncAnimation(
    fig, animate, frames=NUM_FRAMES, interval=FRAME_INTERVAL)

In [ ]:
ani

# 3.2 Schrödinger's equation

In [ ]:
@jit
def nabla_sq_1D(N):
    """
    Function which creates matrix containing the discretized nabla^2 
    operator.
    
    N = number of steps in domain
    """
    diagonals = [[1] * (N - 2) + [0], [1] + [-2] * (N - 2) + [1], [0] + [1] * (N - 2)]
    matrix = diags(diagonals, [-1, 0, 1]).toarray() 
    return matrix

### Infinite potential well

In [ ]:
N = 200 # number of steps
a = 1 # width of well (starts at x = 0)
dx = a/N

# create matrix and find eigenvectors/values
matrix = -0.5 * nabla_sq_1D(N) / (dx ** 2)
eigen = eigs(matrix, k=4, which='SM')

# find solution of certain state
state = 0
eigen_vec = eigen[1][:,state]
eigen_val = eigen[0][state]
print(((state + 1) ** 2 * np.pi ** 2)/ (2 * (a **2)))
print(eigen_val)

# x-axis contains values -a to a in N steps
x_axis = np.linspace(-a, a, N)
sol = np.array([np.sqrt(1 / a) * np.cos((state + 1) * np.pi * x / (2*a)) for x in x_axis])

# normalize
sol = normalize(sol.reshape(sol.shape[0],-1), norm='l2', axis=0).reshape(sol.shape)

# print(np.linalg.norm(sol))

# exact solution
plt.plot(x_axis, abs(eigen_vec) ** 2, label='numerical')
plt.plot(x_axis, abs(sol) ** 2, label='exact',linestyle=':')
plt.xlabel('x')
plt.ylabel('$\Psi$')
plt.legend()

### Finite potential well

In [ ]:
def fin_well_discr(N, p, a, V0):
    """
    Function which creates matrix for finite well problem.
    
    N = number of steps in domain
    V0 is potential in the domain where V != 0 
    
    V = 0 for -a < x < a
    V = V0 for -p < x < -a and a < x < p 
    """
    
    # divide total length in N steps
    dX = 2 * p / N
    
    # boundary between V=V0 and V=0 
    boundary = int(abs(p-a)/dX)
    
    # create matrix 
    haha = -1 + V0
    diagonals = [[-0.5] * (N - 1), [1 + V0] * boundary + [1] * (N - 2 * boundary) + [1 + V0] * boundary, [-0.5] * (N - 1)]
    matrix = diags(diagonals, [-1, 0, 1]).toarray()
    return matrix

N = 100
a = 3
p = 7
V0 = 1
matrix = fin_well_discr(N, p, a, V0)
print(matrix)

state = 0
eigen = eigs(matrix, k=state+1, which='SM')

eigen_vec = eigen[1][:,state]
eigen_val = eigen[0][state]
print(eigen_val)

# x-axis contains values -a to a in N steps
x_axis = np.linspace(-p, p, N)

# plot line for well
plt.plot(x_axis, eigen_vec)
plt.axvline(x=a, c='red')
plt.axvline(x=-a, c='red')
plt.xlabel('x')
plt.ylabel('$\Psi$')

### Parabolic potential well

In [ ]:
def parabolic_well_discr(N, a, b):
    """
    Function which creates matrix for parabolic well problem. 
    
    V = bx^2 (potential, x is position in domain)
    a = range of domain (from -a to a)
    N = number of steps in domain
    """
    
    # divide total domain in N steps
    dX = 2 * a / N
    
    # create middle diagonal (1 + V(x))
    diag = [(b * x ** 2 + 1/(dX ** 2)) for x in np.linspace(-a, a, N)]
    
    # create matrix 
    diagonals = [[-0.5 / (dX ** 2)] * (N - 1), diag, [-0.5 / (dX ** 2)] * (N - 1)]
    matrix = diags(diagonals, [-1, 0, 1]).toarray()
    return matrix

In [ ]:
N = 400
a = 10
b = 1/2
matrix = parabolic_well_discr(N, a, b)
state = 0
eigen = eigs(matrix, k=state+1, which='SM')

eigen_vec = eigen[1][:,state]
eigen_val = eigen[0][state]

# x-axis contains values -a to a in N steps
x_axis = np.linspace(-a, a, N)

#taken from https://gist.github.com/dougmcnally/0725be63973022a55d7cec43649b6691
herm_coeff = [0]*state + [1]
sol = np.exp(-x_axis**2/2) * np.polynomial.hermite.hermval(x_axis, herm_coeff)

# normalize
sol = normalize(sol.reshape(sol.shape[0],-1), norm='l2', axis=0).reshape(sol.shape)

print(eigen_val) # these should be equal for first state...
print((state + 1/2))

print(np.linalg.norm(sol)) # should both be normalized to 1..
print(np.linalg.norm(eigen_vec))

# plot line for well
fig, ax = plt.subplots()
ax.plot(x_axis, abs(eigen_vec ** 2), label='numerical',c='g')
ax.plot(x_axis, abs(sol) ** 2, label='exact',c='b',linestyle=':')

ax1 = ax.twinx()
ax1.plot(x_axis, [b * (x ** 2) for x in x_axis], label='V',c='r')

#https://stackoverflow.com/a/10129461
# ask matplotlib for the plotted objects and their labels
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax1.get_legend_handles_labels()
ax.legend(lines + lines2, labels + labels2,loc=2)

ax.set_xlabel('x')
ax.set_ylabel('normalized $\Psi$')
ax1.set_ylabel('$V=%.2f x^2$'%(b))
ax.set_title("$\Psi$ for a parabolic potential well")

In [ ]:
a = 2
N = 10
dX  = 2 * a / N
print(np.arange(-a, a, dX))
print(np.linspace(-a, a, N))

# 3.3 Direct methods for solving steady state problems

In [ ]:
def diffusion_direct(shape, source, radius):
    y, x = shape
    dx = 2 * radius / y

    M = laplacian(shape, dx)
    b = np.zeros((shape))
    b[int(x / 2 + (x * source[1]) / (2 * radius)),\
      int(y / 2 + (y * source[0]) / (2 * radius))] = 1
    b = b.reshape(x * y)

    #gives negative concentrations?
    c = -spsolve(M, b).reshape(shape)

    fig, ax = plt.subplots()
    im = ax.imshow(c, origin='lower',\
                   extent=[-radius, radius, -radius, radius])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("Diffusion equation using direct methods\nfor a square domain")
    cb = plt.colorbar(im)
    cb.set_label("concentration")
    return c


def diffusion_direct_circle(size, source, radius):
    """
    Solves the time-independent diffusion equation using
    a direct method on a circle. The circle has a certain
    radius and a single source. The (square) space is
    discretized into size*size.
    """
    c = circular(size)
    assert c[int(size / 2 + (size * source[1]) / (2 * radius)),\
      int(size / 2 + (size * source[0]) / (2 * radius))] == 1,\
    "Source location is not part of the circle!"

    dx = 2 * radius / size
    M = laplacian_circle(size, dx)
    b = np.zeros((size, size))
    b[int(size / 2 + (size * source[1]) / (2 * radius)),\
      int(size / 2 + (size * source[0]) / (2 * radius))] = 1
    b = b.reshape(size * size)

    #gives negative concentrations?
    c = -spsolve(M, b).reshape(size, size)

    fig, ax = plt.subplots()
    im = ax.imshow(c, origin='lower',\
                   extent=[-radius, radius, -radius, radius])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title(
        "Diffusion equation using direct methods\nfor a circular domain")
    cb = plt.colorbar(im)
    cb.set_label("concentration")
    return c


diffusion_direct_circle(100, (0.6, 1.2), 2);
# diffusion_direct((100,100),(0.6,1.2),2);